In [1]:
# Получаем список обменников
import requests
from bs4 import BeautifulSoup

r = requests.get("https://www.cointelligence.com/exchanges_list/") 

page = BeautifulSoup(r.text)

exchange_list = []
#Выбираем все названия бирж
data = page.find_all("td", {"class": "name"})
for i in data:
    exchange_list.append(i.contents[2].text.replace("[SCAM Alert]","")) # Убирам скам алерт

In [2]:
# Получаем новости
from rss_parser import Parser
import datetime
#Топики
TOPICS = ["hacker attack","law enforcement","fraud","funds withdrawal"]
#Ивенты которые получим в виде словаря {ecxchange - обменник, topic - топик из списка топриков, date - дата события}
events = []

for exchange in exchange_list:
    now = datetime.datetime.now()
    five_days_ago = now - datetime.timedelta(days=5)
    for topic in TOPICS:
        url = f"http://news.google.com/news?q={exchange}%20{topic}&as_qdr=w&output=rss"
        r = requests.get(url)
        parser = Parser(xml=r.content)
        feed = parser.parse()
        for item in feed.feed:
            # У гугл новостей нет фильтра по датам для RSS. Фильтруем
            event_date = datetime.datetime.strptime(item.publish_date,"%a, %d %b %Y %H:%M:%S %Z")
            if event_date > five_days_ago:
                events.append({"exchange":exchange, "topic":topic, "date":item.publish_date})



In [ ]:
# Стоп слова - отсеиваем сообщения ботов, выдающих статистику

STOP_WORDS = ["alert","24h volume","previous alt", "rewards"]

In [10]:
import re
import time
import tweepy
from textblob import TextBlob
import datetime

# Стоп слова - отсеиваем сообщения ботов, выдающих статистику

STOP_WORDS = ["alert","24h volume","previous alt", "rewards"]

def stop_words_check(text):
    for word in STOP_WORDS:
        if text.find(word)!=-1:
            return True
    return False

barier = "AAAAAAAAAAAAAAAAAAAAAEagDAEAAAAAV%2Bw9NxUw3WDjUg%2FO4ad3WdOjRYc%3DL5tcIoSkdJGUVEnGxmPGTzqTD7LrHraSWS5crTnZHHZ74OWB3d"

client = tweepy.Client(bearer_token=barier)

# Удаяем лики и спец символы
def clean_tweet(tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])(\w+:\/\/\S+)", " ", tweet).split())

result = []
authors = [] #Список авторов чтоб избежать повторений
texts =[]



for event in events:
    if len(result)>250:
        break #Прерываем если достигли нужного количества результатов
    event_date = datetime.datetime.strptime(event["date"],"%a, %d %b %Y %H:%M:%S %Z")
    start_date = event_date - datetime.timedelta(hours=5)
    end_date = event_date+datetime.timedelta(hours=5)
    #ДатаВремя до должно быть на 10 сек меньше даты и времени запроса (на всякий лучай берем 20 сек)
    if end_date > datetime.datetime.utcnow() - datetime.timedelta(seconds=20):
        end_date = datetime.datetime.utcnow() - datetime.timedelta(seconds=20) 
  
    try:
        tweets = client.search_recent_tweets(query=event["exchange"], 
                                        end_time=end_date, 
                                        start_time=start_date,
                                        tweet_fields = ["author_id","text","created_at"]
                                        )
        if tweets.data:
            for tweet in tweets.data:
                #Проверка на стоп слова
                if stop_words_check(tweet.text.lower()):
                    continue
                # Фильтруем чтобы были уникальные автора
                if tweet.author_id in authors:
                    continue

                if tweet.text in texts:
                    continue

                analysis = TextBlob(clean_tweet(tweet.text)) # Сентиментальный анализ
                if (analysis.sentiment.polarity < 0): #Если новость негативная
                    result.append({"author":tweet.author_id,
                                   "date":tweet.created_at,
                                   "exchange":event["exchange"],
                                   "text":tweet.text,

                                   })
                    authors.append(tweet.author_id)
                    texts.append(tweet.text)
    except Exception as e:
        print(e)
        time.sleep(60*15) #Ждем 15 минут если уперлись в лимит               





429 Too Many Requests


In [11]:
# Находим имена юзеров
for i in result:
    user = client.get_user(id=int(i["author"]))
    try:
        i["author_name"] = user.data.username
    except:
        continue


In [14]:
#Сохраняем в csv
with open("res.csv","w") as f:
    for i in result:
        text =i['text'].replace('\n','')
        f.write(f"{i['author_name']};{i['date']};{i['exchange']};{text}\n")
